In [1]:
#COMPARE DEFINITIONS FROM TEXT WORDS TO DEFINITIONS FROM DESCRIPTION WORDS


#the standard
import numpy as np
import pandas as pd

#will clean the emoji descriptions using stop words
import nltk
#nltk.download()
#from nltk import tag
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
StopWords = stopwords.words("english")

#will be doing some punctuation cleaning
import string

#will be using synonyms of words in the emoji descriptions
from PyDictionary import PyDictionary
dictionary=PyDictionary()

In [2]:
#Used word2vec to create VSM
from gensim.models import word2vec

#load W2V VSM model
#model = word2vec.Word2Vec.load_word2vec_format('word2vec/text8.model.bin', binary=True)

googlenews = 'word2vec/GoogleNews-vectors-negative300.bin.gz'
model = word2vec.Word2Vec.load_word2vec_format(googlenews, binary=True)

#testing
#pos = "woman king".lower().split(" ")
#print model.most_similar(positive=pos, negative=['man'], topn=5)

#val = model['woman'] + model['king'] - model['man']  
#print np.dot(val,model["queen"])/( np.linalg.norm(model['queen']) * np.linalg.norm(val) )

#print model.most_similar(positive=val, topn=5)
#print model.similarity(val,'woman')
#print np.sum(model[['woman','king']], axis=0)- model['woman'] - model['king']

In [3]:
def cos_sim(v1,v2):
    x = np.dot(v1,v2)/( np.linalg.norm(v1) * np.linalg.norm(v2) )
    return x

def wdm(v1,v2):
    x = 1./np.linalg.norm(v1-v2)
    if np.isfinite(x):
        return x
    else:
        return 2

In [4]:
#Load emoji data
emoji_key = pd.read_csv('emojis/emojiDict_complete.csv', sep=';')
# print emoji_key.head(5)
print len(emoji_key.Bytes.values)
emoji_key = emoji_key.dropna()
print len(emoji_key.Bytes.values)
print emoji_key.head(5)

842
835
      Description Native             Bytes                R-encoding  \
0  aerial tramway     🚡  \xF0\x9F\x9A\xA1  <ed><a0><bd><ed><ba><a1>   
1        airplane      ✈      \xE2\x9C\x88              <e2><9c><88>   
2     alarm clock      ⏰      \xE2\x8F\xB0              <e2><8f><b0>   
3   alien monster     👾  \xF0\x9F\x91\xBE  <ed><a0><bd><ed><b1><be>   
4       ambulance     🚑  \xF0\x9F\x9A\x91  <ed><a0><bd><ed><ba><91>   

                                         Synonyms  \
0    aerial flying airy tramway trim away dragway   
1                           airplane jet aircraft   
2  alarm nervousness dismay clock alarm timepiece   
3        alien exotic unusual monster giant devil   
4                      ambulance transport rescue   

                                            DefWords  \
0  existing living growing operating air characte...   
1  aircraft fixed wing powered propellers jets bl...   
2  fill apprehension alarm cause unpleasantly sur...   
3  contained derivi

In [5]:
#reset index for datafram
emoji_key = emoji_key.reset_index(drop=True)
print emoji_key[emoji_key['Description']=="chicken"]

    Description Native             Bytes                R-encoding  \
124     chicken     🐔  \xF0\x9F\x90\x94  <ed><a0><bd><ed><b0><94>   

                     Synonyms  \
124  chicken poltroon dastard   

                                              DefWords  \
124  easily frightened characterized complete cowar...   

                                               DescDef  
124  easily frightened flesh chicken used food dome...  


In [6]:
#GET SIMILARITY BASED ON AVERAGED SIMILARITY BETWEEN WORDS IN MODEL
def compare(l1,l2):
    x=0.
    n=0.
    for w1 in l1:
        #print w1
        for w2 in l2:
            #print w1, w2
            #get model similarity for every synonym
            try:
                x += model.similarity(w1, w2)
                n += 1.
            except:
                x += 0.
                n += 0.
            if x==0:
                n=1
            #print x

    return x/n
    
# s1 = "Take bus school".lower().split(" ")
# s2 = emoji_key.DefWords.values[0].split(" ")
# print compare(s1,s2)
# s1 = "Love cake".lower().split(" ")
# s2 = emoji_key.DefWords.values[0].split(" ")
# print compare(s1,s2)

In [7]:
#GET SIMILARITY BASED ON SIMILARITY BETWEEN WORDS VECTOR SUM LOCATIONS IN MODEL
def compare_alt(l1,l2):
    s1 = l1[:]
    s2 = l2[:]
    for w1 in l1:
        try:
            x = model[w1]
        except:
            s1.remove(w1)
    for w2 in l2:
        try:
            x = model[w2]
        except:
            s2.remove(w2)
    
    if len(s1)==0 or len(s2)==0:
        return 0.
    
    v1 = np.sum(model[s1], axis=0)
    v2 = np.sum(model[s2], axis=0)
    
    return cos_sim(v1,v2)

s1 = ['easily','caught']
s2 = emoji_key.Description[124].split(" ")#emoji_key.DescDef[124].split(" ")[:3]
print s2
print s1==s2

print compare_alt(s1,s2)

['chicken']
False
0.137762


In [8]:
#GET SIMILARITY BASED ON SIMILARITY BETWEEN 2 SENTENCES (vector average then take euclidian distance)
def compare_alt1(l1,l2,avg):
    s1 = l1[:]
    s2 = l2[:]
    for w1 in l1:
        try:
            x = model[w1]
        except:
            s1.remove(w1)
    for w2 in l2:
        try:
            x = model[w2]
        except:
            s2.remove(w2)
    
    if len(s1)==0 or len(s2)==0:
        return 0.

    v1 = np.sum(model[s1], axis=0)
    v2 = np.sum(model[s2], axis=0)
    
    if avg==True:
        v1 = v1/float(len(s1))
        v2 = v2/float(len(s2))
        
    #print cos_sim(v1,v2)
    return wdm(v1,v2)

s1 = ['easily','caught']
s2 = emoji_key.Description[124].split(" ")#emoji_key.DescDef[124].split(" ")[:3]
print s2
print s1==s2

print compare_alt1(s1,s2, avg=True)

['chicken']
False
0.313535741344


In [9]:
#Clean the emoji descriptions for easy use later
def Clean(text):
    #REMOVE STOPWORDS
    text = ' '.join([word.lower() for word in text.split() if word not in StopWords])
    #remove punctuation
    text = text.translate(string.maketrans("",""), string.punctuation)
    return text

In [10]:
print emoji_key.values[205][-1]

run move quickly hastily break pieces striking knocking hurl thrust violently destroy break cause lose courage add enlivening altering element distinctive stylish elegance quick run footrace run top speed punctuation mark  longer two telegraphic signals used morse code act moving great haste arbitrary sign written printed something visible association convention represents something else invisible


In [11]:
# sentences = ["watching soccer Love Cake like eat chicken fingers",\
#              "take car downtown shake bake bottled water dinner morning",\
#              "Why can't the rattled galaxy shoot the raw arrogance",\
#              "Can a cupboard rage?",\
#              "at the gym lifting weights"]

sentences = [
    "You had time to call the police",\
    "I am going to marry her",\
    "Are you taking his side against me",\
    "What do you mean, you've lost the lottery ticket",\
    "If you do this, you will be dead to me",\
    "What do you remember about your mother?"
]

# sentences = ["food was tasty","I enjoy having you here"]

NonTypes = ['PRP','PRP$','TO','WP','WP$','IN','DT']

for usrtxt in sentences:
    #usrtxt = Clean(usrtxt) #.split(" ")
    usrtxt = usrtxt.translate(string.maketrans("",""), string.punctuation) #edit
    print "Usr txt:",usrtxt
    usrtxt = word_tokenize(usrtxt)
    usrtxt = nltk.pos_tag(usrtxt)

    emojis = []

    for w in usrtxt:
        #print w
        
        #Check if word is really valid
        if w[1] in NonTypes:
            emojis.append(w[0])
            continue
        else:
            w = w[0]
            
        #get definition of the word in our given sentence
        try:
            dfn = dictionary.meaning(w)
            if dfn==None:
                print "Could not find definition for:",w
                print dfn
                emojis.append(w)
                continue
        except:
            print "Could not find definition for:",w
            continue
        #given definition for word 'w' in list dfn_words
        dfn_words = []
        for key in dfn:
            words = dfn[key]
            words = " ".join(words)
            words = Clean(words).split(" ")
            dfn_words = dfn_words + words
        
        #print dfn_words
        
        #Define a list of similarity values
        similarity = []

        #compare every definition word to every description definition word 
        #print "getting similarities..."
        
    # [X] GET SIMILARITY BASED ON AVERAGE SIMILARITY BETWEEN ALL WORD DEFINITIONS AND EMOJI DEFINITIONS
        #similarity = emoji_key.DescDef.apply(lambda x: compare(x.split(" "), dfn_words) )
    
    #GET SIMILARITY BASED ON VECTOR SUM OF SIMILARITY BETWEEN SUM(WORD DEFINITIONS) AND SUM(EMOJI DEFINITIONS)
        similarity = emoji_key.DescDef.apply(lambda x: compare_alt(x.split(" "), dfn_words) )
    
    #GET SIMILARITY BASED ON VECTOR SUM OF SIMILARITY BETWEEN WORD AND EMOJI DESCRIPTION
        similarity += emoji_key.Description.apply(lambda x: compare_alt(x.split(" "), [w]) )
    
    #GET SIMILARITY BASED ON BETWEEN WORD AND EMOJI DESCRIPTION (vector average and take euclidian distance)
        similarity += emoji_key.Description.apply(lambda x: compare_alt1(x.split(" "), [w], avg=True) )
    
    #GET SIMILARITY BASED ON AVERAGE SIMILARITY BETWEEN WORD AND EMOJI DESCRIPTION SYNONYM WORDS
        similarity += emoji_key.Synonyms.apply(lambda x: compare(x.split(" "), [w]) )
    
    #GET SIMILARITY BASED ON SIMILARITY BETWEEN WORD AND EMOJI DESC. SYNONYM WORDS (vector average and take euclidian distance)
        similarity += emoji_key.Synonyms.apply(lambda x: compare_alt1(x.split(" "), [w], avg=True) )
    
    #GET SIMILARITY BASED ON SIMILARITY BETWEEN WORD SYNONYMS AND EMOJI DESC. SYNONYMS (vector average and take euclidian distance)
        #w_syns = [w]+dictionary.synonym(w)[:2]
        #similarity = emoji_key.Synonyms.apply(lambda x: compare_alt1(x.split(" "), w_syns, avg=True) )
    
    #GET SIMILARITY BASED ON SIMILARITY BETWEEN ALL WORD DEFS AND EMOJI DEFS (vector sum and take euclidian distance)
        #similarity = emoji_key.DescDef.apply(lambda x: compare_alt1(x.split(" "), dfn_words, avg=False) )
        
        tops = sorted(zip(similarity, range(len(similarity))), reverse=True)[:3]
        #print tops
        print w+":"
        for x in tops:
            print emoji_key.Native.values[x[1]],\
            emoji_key.Description.values[x[1]],\
            similarity[x[1]]

        emojis.append(emoji_key.Native.values[tops[0][1]])

    for i in emojis:
        print i
    print "\n"
#     if max(match)>0.5:
#         print emoji_key.Native.values[match.index(max(match))],\
#         emoji_key.Description.values[match.index(max(match))],\
#         max(match)
#     else:
#         print "no match >0.8"

Usr txt: You had time to call the police
had:
😥 disappointed but relieved face 2.39211500238
💑 couple with heart 2.37536911627
🌜 last quarter moon with face 2.34546671598
time:
🌛 first quarter moon with face 2.53346713831
🌜 last quarter moon with face 2.48977340708
🕕 clock face six oclock 2.4344607546
call:
☎ black telephone 2.20662805239
📱 mobile phone 2.19044418922
📴 mobile phone off 2.15383582704
police:
👮 police officer 3.30456808557
🚓 police car 3.00805129481
🚔 oncoming police car 2.58604385977
You
😥
🌛
to
☎
the
👮


Usr txt: I am going to marry her
am:
🌉 bridge at night 1.96661203975
🕐 clock face one oclock 1.90043389298
🚯 do not litter symbol 1.86950219636
going:
🙋 happy person raising one hand 2.52042287833
😥 disappointed but relieved face 2.48821487658
🚯 do not litter symbol 2.46780593089
marry:
💒 wedding 2.7084891554
👰 bride with veil 1.67534462706
👩 woman 1.58610221403
I
🌉
🙋
to
💒
her


Usr txt: Are you taking his side against me
Are:
😫 tired face 1.80067889854
🙈 see no evil mo

C:\Users\Anael\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
text = word_tokenize("They refuse to permit us to obtain the refuse permit")
text = word_tokenize("I like to eat chicken like cows do")
text = word_tokenize("I take what like If you love to do this love")
tags = nltk.pos_tag(text)
print tags
print tags[0][1]

[('I', 'PRP'), ('take', 'VBP'), ('what', 'WP'), ('like', 'IN'), ('If', 'IN'), ('you', 'PRP'), ('love', 'VBP'), ('to', 'TO'), ('do', 'VB'), ('this', 'DT'), ('love', 'VB')]
PRP
